<a href="https://colab.research.google.com/github/Seandaytoy1005/MMO/blob/main/lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируйте библиотеки, необходимые для предварительной обработки набора данных, разработки функций и обучения модели.

In [1]:

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [26]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
# from sklearn.datasets import load_iris, load_boston
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")

В этой статье я использую набор обзорных данных Amazon.

In [3]:
#加载数据集
data = open('/content/corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
  content = line.split()
  labels.append(content[0])
  texts.append(content[1])

#创建一个dataframe，列名为text和label
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [4]:
print(trainDF)

             text       label
0         Stuning  __label__2
1             The  __label__2
2       Amazing!:  __label__2
3       Excellent  __label__2
4       Remember,  __label__2
...           ...         ...
9995            A  __label__2
9996        Great  __label__2
9997  Interesting  __label__1
9998        Don't  __label__1
9999    Beautiful  __label__2

[10000 rows x 2 columns]


Затем мы разделяем набор данных на обучающий и проверочный наборы, чтобы можно было обучать и тестировать классификатор. Кроме того, мы закодируем наш целевой столбец, чтобы его можно было использовать в моделях машинного обучения:

In [5]:
#将数据集分为训练集和验证集
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label编码为目标变量
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

Далее следует разработка объектов. На этом этапе исходные данные будут преобразованы в векторы объектов, а новые объекты будут созданы на основе существующих данных. Чтобы выбрать важные признаки из набора данных, существует несколько способов:
* Считать векторы объектами.
* Вектор TF-IDF как признак
* Встраивание слов как функции
* Функции на основе текста/НЛП
* Тематические модели как функции

CountVectorizer

In [7]:

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

#使用向量计数器对象转换训练集和验证集
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)



Векторы TF-IDF как признаки

Использование [N-грамм](https://ru.wikipedia.org/wiki/N-%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0)

В компьютерной лингвистике существует гипотеза о том, что основными носителями смысла в предложении являются не слова, а словосочетания. Поэтому существует возможность объединения соседних слов в N-граммы.

In [9]:
#词语级tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram 级tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

#词性级tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x)
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:558: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn(



встраивание слов

Встраивание слов — это форма использования плотных векторов для представления слов и документов. Положение слова в векторном пространстве определяется из контекста слова в тексте. Встраивание слов можно обучить с использованием самого входного корпуса или сгенерировать с использованием предварительно обученной модели встраивания слов. Модели встраивания слов включают в себя: Перчатка, FastText, Word2Vec

In [41]:
import gensim
from gensim.models import word2vec

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
model_path = '/content/wiki-news-300d-1M.vec.zip'

In [37]:

embeddings_index = {}
for i, line in enumerate(open('/content/drive/MyDrive/研一/mmo/wiki-news-300d-1M.vec')):
  values = line.split()
  embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [50]:
print("Количество загруженных векторов встраивания слов:", len(embeddings_index))
print("Количество индексов слов для токенизатора:", len(word_index))
print("Создана форма карты встраивания сегментации слов.:", embedding_matrix.shape)

Количество загруженных векторов встраивания слов: 3620
Количество индексов слов для токенизатора: 2586
Создана форма карты встраивания сегментации слов.: (2587, 300)


In [43]:
from gensim.models import Word2Vec
import pandas as pd

data = open('/content/corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(content[1])

trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

model = Word2Vec([text.split() for text in texts], vector_size=300, window=5, min_count=1)

embeddings_index = {word: model.wv[word] for word in model.wv.index_to_key}


token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [51]:
print("Параметры модели:", model)
print("Количество слов в токенайзере:", len(token.word_index))

Параметры модели: Word2Vec<vocab=3620, vector_size=300, alpha=0.025>
Количество слов в токенайзере: 2586


Используйте наивный байесовский классификатор (MultinomialNB) для обучения и прогнозирования функций, а также расчета точности модели.

In [49]:
import pandas as pd
from sklearn import model_selection, preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB()
clf.fit(xtrain_count, train_y)

predictions = clf.predict(xvalid_count)
accuracy = accuracy_score(valid_y, predictions)

print("Accuracy ( CountVectorizer） of the model: {:.2f}%".format(accuracy * 100))


Accuracy ( CountVectorizer） of the model: 69.84%


In [48]:

clf_word = MultinomialNB()
clf_word.fit(xtrain_tfidf, train_y)
predictions_word = clf_word.predict(xvalid_tfidf)
accuracy_word = accuracy_score(valid_y, predictions_word)

clf_ngram = MultinomialNB()
clf_ngram.fit(xtrain_tfidf_ngram, train_y)
predictions_ngram = clf_ngram.predict(xvalid_tfidf_ngram)
accuracy_ngram = accuracy_score(valid_y, predictions_ngram)

clf_chars = MultinomialNB()
clf_chars.fit(xtrain_tfidf_ngram_chars, train_y)
predictions_chars = clf_chars.predict(xvalid_tfidf_ngram_chars)
accuracy_chars = accuracy_score(valid_y, predictions_chars)

print("Accuracy (word-level tf-idf): {:.2f}%".format(accuracy_word * 100))
print("Accuracy (ngram-level tf-idf): {:.2f}%".format(accuracy_ngram * 100))
print("Accuracy (character-level tf-idf): {:.2f}%".format(accuracy_chars * 100))

Accuracy (word-level tf-idf): 69.76%
Accuracy (ngram-level tf-idf): 50.44%
Accuracy (character-level tf-idf): 67.08%
